In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math, warnings, os


In [6]:
# === 1. Carregar base ===
df = pd.read_excel("../Data/base_vendas_pdv_IA.xlsx", sheet_name="Sheet1")
df = df.sort_values(["PDV_ID", "Data"]).reset_index(drop=True)
df["Ano"] = df["Data"].dt.year
df["Mes"] = df["Data"].dt.month

In [7]:

# === 2. Definir features e target ===
features = ["Custo", "Lucro", "Satisfacao", "Mes", "Ano", "Selic", "Inflacao", "Qtd_Cotacoes"]
target = "Vendas"
SEQ_LEN = 48
# === 3. Limpeza geral de dados ===
# Substitui infs e remove linhas com NaN em features/target
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=features + [target])
print(f"✅ Base limpa: {df.shape[0]} linhas restantes após remoção de NaN/infs.")

✅ Base limpa: 6063 linhas restantes após remoção de NaN/infs.


In [9]:
# === 4. Funções auxiliares ===
def make_sequences(data, seq_len, feature_cols, target_col):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[feature_cols].iloc[i:i+seq_len].values)
        y.append(data[target_col].iloc[i+seq_len])
    return np.array(X), np.array(y)

def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mae")
    return model

In [10]:
# === 5. Loop por PDV ===
predicoes_futuras = []
metricas_modelos = []
scalers = {}

for pdv_id, grupo in df.groupby("PDV_ID"):
    if len(grupo) < SEQ_LEN + 8:
        print(f"⚠️ PDV {pdv_id} ignorado (histórico curto).")
        continue

    g = grupo.copy().reset_index(drop=True)

    # Se ainda restarem NaNs nesse PDV, pula
    if g[features + [target]].isnull().any().any():
        print(f"⚠️ PDV {pdv_id} ignorado (NaN detectado após limpeza).")
        continue

    # === Escalonamento ===
    feature_cols = features + [target]
    scaler = MinMaxScaler()
    g_scaled = pd.DataFrame(scaler.fit_transform(g[feature_cols]), columns=feature_cols)
    scalers[pdv_id] = scaler

    # === Sequências ===
    X, y = make_sequences(g_scaled, SEQ_LEN, features, target)
    if len(X) == 0:
        print(f"⚠️ PDV {pdv_id}: não gerou sequências suficientes.")
        continue

    # === Split treino/teste ===
    split = int(len(X) * 0.85)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Check final: NaN nas matrizes?
    if np.isnan(X_train).any() or np.isnan(y_train).any():
        print(f"⚠️ PDV {pdv_id}: contém NaN no treino, ignorado.")
        continue

    # === Modelo ===
    model = build_lstm_model((SEQ_LEN, len(features)))
    early = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=80,
        batch_size=16,
        verbose=0,
        callbacks=[early]
    )

    # === Avaliação ===
    y_pred = model.predict(X_test).flatten()
    y_test_inv = scaler.inverse_transform(
        np.concatenate([np.zeros((len(y_test), len(features))), y_test.reshape(-1,1)], axis=1)
    )[:,-1]
    y_pred_inv = scaler.inverse_transform(
        np.concatenate([np.zeros((len(y_pred), len(features))), y_pred.reshape(-1,1)], axis=1)
    )[:,-1]

    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    rmse = math.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    r2 = r2_score(y_test_inv, y_pred_inv)
    metricas_modelos.append([pdv_id, mae, rmse, r2])

    # === 6. Previsão futura de 8 meses ===
    ultimos_dados = g_scaled[features].iloc[-SEQ_LEN:].values
    preds_futuras = []

    for step in range(8):
        X_in = ultimos_dados.reshape(1, SEQ_LEN, len(features))
        y_pred = model.predict(X_in, verbose=0).flatten()[0]

        nova_linha = ultimos_dados[-1].copy()
        nova_linha[features.index("Mes")] = ((g.iloc[-1]["Mes"] + step + 1 - 1) % 12) / 12
        nova_linha[features.index("Ano")] = (g.iloc[-1]["Ano"] - df["Ano"].min()) / (df["Ano"].max() - df["Ano"].min())

        ultimos_dados = np.vstack([ultimos_dados[1:], nova_linha])
        preds_futuras.append(y_pred)

    # Inverter escala das previsões
    preds_futuras_inv = scaler.inverse_transform(
        np.concatenate([np.zeros((len(preds_futuras), len(features))), np.array(preds_futuras).reshape(-1,1)], axis=1)
    )[:,-1]

    df_pred = pd.DataFrame({
        "PDV_ID": pdv_id,
        "Mes_Futuro": range(1, 9),
        "Vendas_Previstas": preds_futuras_inv
    })
    predicoes_futuras.append(df_pred)


C:\Users\José Orlei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


C:\Users\José Orlei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


C:\Users\José Orlei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step


C:\Users\José Orlei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


C:\Users\José Orlei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


In [11]:

# === 7. Consolidação ===
df_futuro = pd.concat(predicoes_futuras, ignore_index=True)
df_metricas = pd.DataFrame(metricas_modelos, columns=["PDV_ID", "MAE", "RMSE", "R²"])

df_futuro.to_excel("previsao_lstm_8_meses_por_pdv.xlsx", index=False)
df_metricas.to_excel("metricas_lstm_por_pdv.xlsx", index=False)

print("\n✅ LSTM treinado com sucesso!")
print("📈 previsao_lstm_8_meses_por_pdv.xlsx")
print("📊 metricas_lstm_por_pdv.xlsx")



✅ LSTM treinado com sucesso!
📈 previsao_lstm_8_meses_por_pdv.xlsx
📊 metricas_lstm_por_pdv.xlsx
